In [1]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tfio

2023-04-07 18:16:15.801978: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
NUM_COLUMNS = 18
BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64

def decode_kafka_item(item):
  message = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(item.key)
  return (message, key)

train_ds = tfio.IODataset.from_kafka("tf.public.susy_train", partition=0, offset=0, servers="kafka:9092")
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

2023-04-07 18:17:06.987829: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: bootstrap.servers=kafka:9092
2023-04-07 18:17:08.504187: I tensorflow_io/core/kernels/kafka_kernels.cc:349] Kafka tail: 4820


In [4]:
OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(NUM_COLUMNS,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [5]:
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10


2023-04-07 18:17:55.280463: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_14' with dtype resource
	 [[{{node Placeholder/_14}}]]
2023-04-07 18:17:55.280947: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [1]
	 [[{{node Placeholder/_10}}]]
/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


      1/Unknown - 1s 927ms/step - loss: 0.7444 - accuracy: 0.4688

2023-04-07 18:17:56.136187: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 0
2023-04-07 18:17:56.279117: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 1024


     30/Unknown - 1s 18ms/step - loss: 0.6425 - accuracy: 0.6089

...

42/76 [===============>..............] - ETA: 0s - loss: 0.4362 - accuracy: 0.8017

2023-04-07 18:18:20.253169: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 3072


57/76 [=====================>........] - ETA: 0s - loss: 0.4451 - accuracy: 0.7980

2023-04-07 18:18:20.765487: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 4096


76/76 [==============================] - 3s 29ms/step - loss: 0.4471 - accuracy: 0.7936


In [6]:
test_ds = tfio.IODataset.from_kafka("tf.public.susy_test", partition=0, offset=0, servers="kafka:9092")
test_ds = test_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
test_ds = test_ds.map(decode_kafka_item)
test_ds = test_ds.batch(BATCH_SIZE)

2023-04-07 18:18:31.790434: I tensorflow_io/core/kernels/kafka_kernels.cc:107] Kafka configuration: bootstrap.servers=kafka:9092
2023-04-07 18:18:32.299099: I tensorflow_io/core/kernels/kafka_kernels.cc:349] Kafka tail: 5000


In [7]:
res = model.evaluate(test_ds)
print("test loss, test acc:", res)

2023-04-07 18:18:42.601898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [1]
	 [[{{node Placeholder/_10}}]]
2023-04-07 18:18:42.602594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype int64 and shape [1]
	 [[{{node Placeholder/_10}}]]
2023-04-07 18:18:42.774112: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 0


      1/Unknown - 0s 253ms/step - loss: 0.4685 - accuracy: 0.8281

2023-04-07 18:18:42.885613: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 1024


     16/Unknown - 1s 34ms/step - loss: 0.4229 - accuracy: 0.8115 

2023-04-07 18:18:43.405624: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 2048


     43/Unknown - 1s 25ms/step - loss: 0.4462 - accuracy: 0.8001

2023-04-07 18:18:43.936422: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 3072


     56/Unknown - 2s 29ms/step - loss: 0.4486 - accuracy: 0.8002

2023-04-07 18:18:44.473210: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 4096


79/79 [==============================] - 2s 27ms/step - loss: 0.4459 - accuracy: 0.8020
test loss, test acc: [0.4458748996257782, 0.8019999861717224]
